In [2]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2
import math
import sys
from pathlib import Path

sys.path.append('/media/data/robert/code/bids')
from bids_files import BIDS_FILE
import numpy as np
from nii_utils import calc_centroids, centroids_to_dict
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from sitk_utils import affine_registration_transform,apply_transform


import open3d as o3d

subreg_nib: nib.Nifti1Image = nib.load("vertebra_align/subreg.nii.gz")
vert_nib = nib.load("vertebra_align/vert.nii.gz")
ct_nib = nib.load("vertebra_align/ct.nii.gz")
subreg_arr1 = subreg_nib.get_fdata()
vert_arr1 = vert_nib.get_fdata()
arr = np.zeros_like(subreg_arr1)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
vert_arr_dict:dict[int,np.ndarray] = {}
for vert_id in np.unique(vert_arr1):
    # vert_id = 17
    print(vert_id)
    subreg_arr = subreg_arr1.copy()
    vert_arr = vert_arr1.copy()
    vert_arr[vert_arr != vert_id] = 0
    vert_arr[vert_arr == vert_id] = 1
    subreg_arr *= vert_arr
    vert_arr_dict[vert_id] = subreg_arr

0.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0


In [7]:
from optparse import Option
from typing import Tuple


def single_out_vertebra(idx:int, seg: np.ndarray, img)-> Option[sitk.Image]:
    if isinstance(seg,dict) :
        seg = seg[idx]
    else:
        seg = seg.copy()
        seg[seg!=idx] = 0
    inp = np.array(np.where(seg!=0))
    if inp.shape[-1] == 0:
        return None
    # Get smallest rotated 3d BBox
    sub_pcd_o3d = o3d.utility.Vector3dVector(inp.T)
    o3d_bbox = o3d.geometry.OrientedBoundingBox.create_from_points(sub_pcd_o3d)
    
    extent = list(reversed([int(round(i)) for i in o3d_bbox.extent]))
    #image_2D = sitk.Image(seg.shape, sitk.sitkFloat32)
    image_2D = sitk.GetImageFromArray(seg)
    #image_2D[extent[0]-2:extent[0],extent[1]-2:extent[1],extent[2]-2:extent[2]] = 1
    
    translation_center = sitk.TranslationTransform(3, list(reversed(o3d_bbox.center)))
    translation_extend = sitk.TranslationTransform(3, [i/2 for i in reversed(o3d_bbox.extent)])
    rotation = sitk.VersorTransform()
    rotation.SetCenter([-i for i in reversed(o3d_bbox.center)])
    rotation.SetMatrix(np.array(o3d_bbox.R[::-1,::-1]).reshape(-1))
    print(o3d_bbox)

    composite = sitk.CompositeTransform(sitk.Transform())
    composite.AddTransform(translation_center)
    composite.AddTransform(rotation)
    composite.AddTransform(translation_extend.GetInverse())
      
    image_2D = sitk.Resample(
            image_2D,
            sitk.GetImageFromArray(np.zeros(list(reversed(extent)))),
            composite,
            sitk.sitkNearestNeighbor,
            0.0,
            image_2D.GetPixelID(),
        )
    
    
    out = sitk.Round(image_2D) # sitk.GetArrayFromImage(sitk.Cast(sitk.Round(image_2D), sitk.sitkInt32))
    image_2D = sitk.GetImageFromArray(img)
    image_2D = sitk.Resample(
            image_2D,
            sitk.GetImageFromArray(np.zeros(list(reversed(extent)))),
            composite,
            sitk.sitkLinear,
            0.0,
            image_2D.GetPixelID(),
        )
    out2 = image_2D #sitk.GetArrayFromImage(image_2D)

    return out,out2
    #arr[out!=0] = vert_id

#out, out2 = single_out_vertebra(20,vert_arr_dict,ct_nib.get_fdata())
#nib.save(
#    nib.Nifti1Image(out2, subreg_nib.affine),
#    "vertebra_align/ct2.nii.gz"
#)
#nib.save(
#    nib.Nifti1Image(out, subreg_nib.affine),
#    "vertebra_align/centroids2.nii.gz"
#)

In [112]:
from sitk_utils import *
from nii_utils import calc_centroids
#Select 2 BBoxed Vertebra
print('bbox')
sg_18, ct_18 = single_out_vertebra(18,vert_arr_dict,ct_nib.get_fdata())
sg_20, ct_20 = single_out_vertebra(20,vert_arr_dict,ct_nib.get_fdata())
def get_centroid(sitk_img, id = 50):
    sg_arr = sitk.GetArrayFromImage(sitk_img)
    sg_arr[sg_arr != id] = 0
    return calc_centroids(nib.Nifti1Image(sg_arr,subreg_nib.affine))[1][1:]
print(get_centroid(sg_18),get_centroid(sg_20))
#Register them
print('register')

a = sitk.Euler3DTransform(get_centroid(sg_18))
a.SetTranslation([a-b for a,b in zip(get_centroid(sg_18),get_centroid(sg_20))])

transform = affine_registration_transform(sg_18,sg_18,a)
#transform = affine_registration_exhaustive_transform(sg_18,sg_20, a)

#print(transform)
#Apply transforms
print('transform')
trans_18 = apply_transform(sg_18,sg_20,transform,is_segmentation=True)

nib.save(
    nib.Nifti1Image(sitk.GetArrayFromImage(sg_18), subreg_nib.affine),
    "vertebra_align/ct18.nii.gz"
)
nib.save(
    nib.Nifti1Image(sitk.GetArrayFromImage(sg_20), subreg_nib.affine),
    "vertebra_align/ct20.nii.gz"
)

bbox
[86, 206, 69, 243, 112, 234]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [122]:
from optparse import Option
from typing import Tuple

def get_centroid(arr, id = 50):
    if isinstance(arr,sitk.Image):
        arr = sitk.GetArrayFromImage(arr)
    else:
        arr = arr.copy()
    arr[arr != id] = 0
    return calc_centroids(nib.Nifti1Image(arr,subreg_nib.affine),decimals=100)[1][1:]


def bbox2_nd(img):
	N = img.ndim
	out = []
	for ax in itertools.combinations(reversed(range(N)), N - 1):
		nonzero = np.any(img, axis=ax)
		out.extend(np.where(nonzero)[0][[0, -1]])
	return list(out)


    #inp = np.array(np.where(seg!=0))
    #if inp.shape[-1] == 0:
    #    return None
sg_18, c18 = single_out_vertebra(18,vert_arr_dict)
sg_20, c20 = single_out_vertebra(15,vert_arr_dict)
transform = sitk.Euler3DTransform(get_centroid(sg_18))
transform.SetTranslation([b-a for a,b in zip(c18,c20)])
arr = sg_20.copy()
#arr[arr != 50] = 0
arr2 = sg_18.copy()
#arr2[arr2 != 50] = 0
transform = affine_registration_transform(arr,arr2,transform)
#print(transform)
trans_20 = apply_transform(sg_20,sg_18,transform,is_segmentation=True)


nib.save(
    nib.Nifti1Image(sitk.GetArrayFromImage(trans_20), subreg_nib.affine),
    "vertebra_align/trans_20.nii.gz"
)

nib.save(
    nib.Nifti1Image(sg_18, subreg_nib.affine),
    "vertebra_align/ct18.nii.gz"
)

nib.save(
    nib.Nifti1Image(sg_20, subreg_nib.affine),
    "vertebra_align/ct20.nii.gz"
)

[86, 206, 69, 243, 112, 234]
[74, 211, 71, 232, 293, 399]
Final metric value: -0.2119966113744441
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 9.
itk::simple::Transform
 CompositeTransform (0x55ff78aa00f0)
   RTTI typeinfo:   itk::CompositeTransform<double, 3u>
   Reference Count: 1
   Modified Time: 44648644
   Debug: Off
   Object Name: 
   Observers: 
     none
   Transforms in queue, from begin to end:
   >>>>>>>>>
   Euler3DTransform (0x55ff7c8b16a0)
     RTTI typeinfo:   itk::Euler3DTransform<double>
     Reference Count: 1
     Modified Time: 44648485
     Debug: Off
     Object Name: 
     Observers: 
       none
     Matrix: 
       0.990233 -0.100322 0.0968208 
       0.101672 0.994776 -0.00909918 
       -0.0954022 0.0188543 0.99526 
     Offset: [8.96181, -6.71963, 11.6128]
     Center: [81.3669, 136.199, 92.6028]
     Translation: [3.46922, -0.000939878, 5.97922]
     Inverse: 
       0.990233 0.101672 -0.09540

In [8]:
import itertools
from __future__ import annotations

%load_ext autoreload
%autoreload 2
import math
import sys
from pathlib import Path

sys.path.append('/media/data/robert/code/bids')
from bids_files import BIDS_FILE
import numpy as np
from nii_utils import calc_centroids, centroids_to_dict
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
#import SimpleITK as sitk
#from sitk_utils import affine_registration_transform,apply_transform
import ants

subreg_nib  = nib.load("vertebra_align/subreg.nii.gz")
vert_nib  = nib.load("vertebra_align/vert.nii.gz")
ct_nib  = nib.load("vertebra_align/ct.nii.gz")


arr = vert_nib.get_fdata().copy()
arr[arr != 0] += 1
vert_nib2  = nib.Nifti1Image(arr,vert_nib.affine)
arr2 = subreg_nib.get_fdata().copy()
arr2[arr == 0] = 0
subreg_nib2  = nib.Nifti1Image(arr2,vert_nib.affine)
#vert_nib2  = nib.load("vertebra_align/vert.nii.gz")
ct_nib2  = nib.load("vertebra_align/ct.nii.gz")

def compute_bbox(img:np.ndarray, pad=20):
    N = img.ndim
    out = []
    for ax in itertools.combinations(reversed(range(N)), N - 1):
        nonzero = np.any(img, axis=ax)
        out.extend(np.where(nonzero)[0][[0, -1]])
    bbox = list(out)
    for n in range(N):
        bbox[n*2+0] = max(0,bbox[n*2]-pad)
        bbox[n*2+1] = min(img.shape[n],bbox[n*2+1]+pad)
    return bbox

def compute_vertebra_bbox(vert_idx:int, seg: np.ndarray,imgs, pad=20)-> Option[sitk.Image]:
    if isinstance(seg,nib.Nifti1Image):
        seg = seg.get_fdata().copy()
        imgs = [i.get_fdata().copy() for i in imgs]
    else:
        seg = seg.copy()
        imgs = [i.copy() for i in imgs]
    seg[seg!=vert_idx] = 0
    bbox = compute_bbox(seg)
    seg = seg[bbox[0]:bbox[1],bbox[2]:bbox[3],bbox[4]:bbox[5]]
    imgs = [i[bbox[0]:bbox[1],bbox[2]:bbox[3],bbox[4]:bbox[5]] for i in imgs]
        #center = get_centroid(seg)
    #center = reversed(center)
    return seg, *imgs

def get_centroid(subregion_nii, id = 50):
    #if isinstance(subregion_nii,sitk.Image):
    #    subregion_nii = sitk.GetArrayFromImage(subregion_nii)
    #else:
    subregion_nii = subregion_nii.copy()
    subregion_nii[subregion_nii != id] = 0
    if subregion_nii.sum() == 0:
        return None
    return calc_centroids(nib.Nifti1Image(subregion_nii,subreg_nib.affine),decimals=100)[1][1:]
def dice(im1, im2):
    im1 = np.asarray(im1).astype(bool)
    im2 = np.asarray(im2).astype(bool)
    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")
    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)
    q = im1.sum() + im2.sum()
    if q == 0:
        return 1
    return 2.0 * intersection.sum() / q

def register_vertebras(subreg_nib: nib.Nifti1Image,vert_nib: nib.Nifti1Image,ct_nib: nib.Nifti1Image,subreg_nib2: nib.Nifti1Image,vert_nib2: nib.Nifti1Image,ct_nib2: nib.Nifti1Image,):
    for vert_idx in np.unique(vert_nib.get_fdata().astype(int))[4:]:
        if vert_idx == 0:
            continue
        print('[*] Test vertebra idx ', vert_idx)
        if vert_idx not in list(np.unique(vert_nib2.get_fdata().astype(int))):
            print('[*] Dice 0.00 (no match)')
            continue
        vert1,sub1,ct1 = compute_vertebra_bbox(vert_idx,vert_nib,[subreg_nib,ct_nib])
        vert2,sub2,ct2 = compute_vertebra_bbox(vert_idx,vert_nib2,[subreg_nib2,ct_nib2])

        sub1 = sub1*vert1/vert_idx
        sub2 = sub2*vert2/vert_idx
        
        centroid1 = get_centroid(sub1)
        centroid2 = get_centroid(sub2)
        
        if centroid1 is None:
            print('[*] Skip, reference vertebra centroid not inside image')
            continue
        if centroid2 is None:
            print('[*] Dice 0.00 (no centroid match)')
            continue
        
        #sub1 = sub1.astype(np.uint32)
        #sub2 = sub2.astype(np.uint32)
        fixed = ants.from_numpy(sub1,origin=centroid1)
        moving = ants.from_numpy(sub2,origin=centroid2)
        reg_out = ants.registration(fixed=fixed, moving=moving, type_of_transform="Rigid",interpolator="nearestNeighbor")
        #warped_moving = reg_out['warpedmovout'].numpy()
        warped_moving = ants.apply_transforms(fixed=fixed,moving=moving,transformlist=reg_out['fwdtransforms'],interpolator="nearestNeighbor")
        warped_moving = warped_moving.numpy()
        
        nib.save(
            nib.Nifti1Image(sub1.round(decimals=0), subreg_nib.affine),
            f"vertebra_align/{vert_idx}_o.nii.gz"
        )
        print(np.unique(warped_moving))
        nib.save(
            nib.Nifti1Image(warped_moving, subreg_nib.affine),
            f"vertebra_align/{vert_idx}_r.nii.gz"
        )
        #nib.save(
        #    nib.Nifti1Image(vert2, subreg_nib.affine),
        #    f"vertebra_align/{vert_idx}_u.nii.gz"
        #)
        #nib.save(
        #    nib.Nifti1Image(sub1, subreg_nib.affine),
        #    f"vertebra_align/{vert_idx}_u2.nii.gz"
        #)
        #nib.save(
        #    nib.Nifti1Image(sub2, subreg_nib.affine),
        #    f"vertebra_align/{vert_idx}_o2.nii.gz"
        #)
        
        vert1 = vert1.copy()
        vert1[vert1!=0] = 1
        warped_moving[warped_moving!=0] = 1
        d = dice(vert1,warped_moving)
        print(f'[*] Dice {d:.2f}')
        



        
register_vertebras(subreg_nib,vert_nib,ct_nib,subreg_nib2,vert_nib2,ct_nib2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[*] Test vertebra idx  16
[ 0. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]
[*] Dice 0.87
[*] Test vertebra idx  17
[ 0. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]
[*] Dice 0.85
[*] Test vertebra idx  18
[ 0. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]
[*] Dice 0.83
[*] Test vertebra idx  19
[ 0. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]
[*] Dice 0.63
[*] Test vertebra idx  20
[ 0. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]
[*] Dice 0.60
[*] Test vertebra idx  21
[ 0. 41. 43. 44. 45. 46. 49. 50.]
[*] Dice 0.91
